<a href="https://colab.research.google.com/github/OrionXV/WebtopAppForHateDetection/blob/main/Model/Notebook/HateSpeechModelCreator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install pytorch_lightning 
!pip install transformers
!pip install contractions

     |████████████████████████████████| 584 kB 4.1 MB/s 
     |████████████████████████████████| 596 kB 33.0 MB/s 
     |████████████████████████████████| 136 kB 37.7 MB/s 
     |████████████████████████████████| 409 kB 36.4 MB/s 
     |████████████████████████████████| 1.1 MB 27.6 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 271 kB 39.3 MB/s 
     |████████████████████████████████| 144 kB 40.6 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 4.0 MB 5.4 MB/s 
     |████████████████████████████████| 880 kB 34.5 MB/s 
     |████████████████████████████████| 6.6 MB 30.8 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=c2868dc0b09ddfa165a01e6e20843ff49221026b144528ca222ea731bd18134f
 

In [ ]:
#https://drive.google.com/file/d/14eEuuUnKI2wjrG7oYu2GpeaCdGz6PkCy/view?usp=sharing
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=14eEuuUnKI2wjrG7oYu2GpeaCdGz6PkCy' -O trial.zip 

--2022-05-07 06:18:55--  https://docs.google.com/uc?export=download&id=14eEuuUnKI2wjrG7oYu2GpeaCdGz6PkCy
Resolving docs.google.com (docs.google.com)... 142.250.148.100, 142.250.148.138, 142.250.148.113, ...
Connecting to docs.google.com (docs.google.com)|142.250.148.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-a0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uo418od0rrkaad2hn18ctukeg58sqkd8/1651904325000/04989237414475792611/*/14eEuuUnKI2wjrG7oYu2GpeaCdGz6PkCy?e=download [following]
--2022-05-07 06:18:57--  https://doc-10-a0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/uo418od0rrkaad2hn18ctukeg58sqkd8/1651904325000/04989237414475792611/*/14eEuuUnKI2wjrG7oYu2GpeaCdGz6PkCy?e=download
Resolving doc-10-a0-docs.googleusercontent.com (doc-10-a0-docs.googleusercontent.com)... 209.85.145.132, 2607:f8b0:4001:c1e::84
Connecting to doc-10-a0-docs.googleusercontent.com (doc-10-a0-docs.

In [ ]:
!unzip -q trial.zip 

In [ ]:
import numpy as np
import pandas as pd
import random
import seaborn as sns  
sns.set()
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

%config InLineBackend.figure_format = 'retina'
%matplotlib inline

import csv
import json
import logging

from pathlib import Path
import os
import glob

import tarfile
import tempfile

from tqdm import tqdm
import torch                    
import torchvision
import pytorch_lightning as pl
from torchmetrics import functional as FM 
from sklearn.model_selection import train_test_split

In [ ]:
path = '/content'
extension = 'csv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))
print(result)

['labeled_data.csv']


In [ ]:
data_path = path + '/' + result[0]

In [ ]:
data = pd.read_csv(data_path, usecols = ['class', 'tweet'])
data.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
data.rename(columns={'tweet' : 'text',
                   'class' : 'label'}, 
                    inplace=True)

In [ ]:
#0 = hate speech, 1 = offensive lang, 2 = neither
data.groupby('label').count()

,text
label,
0,1430
1,19190
2,4163


In [ ]:
data.label

0        2
1        1
2        1
3        1
4        1
        ..
24778    1
24779    2
24780    1
24781    1
24782    2
Name: label, Length: 24783, dtype: int64

In [ ]:
import re
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy
import contractions

def textCleaner(mystring):
    mystring = " ".join(mystring.split())
    nystring = contractions.fix(mystring)
    #re.sub(r'(!|.)1+', '', text1) 
    return re.sub(r"^\W+", "", mystring)

def textCleaner_adv(mystring):
    newstring = " ".join(x.lower() for x in mystring.split())
    
    newstring = newstring.replace('[^\w\s]','')
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    newstring = emoji_pattern.sub(r'', newstring)
    
    stop = stopwords.words('english')
    newstring = " ".join(x for x in newstring.split() if x not in stop)
    
    nlp = spacy.load('en', disable=['parser', 'ner'])
    doc = nlp(newstring)

    return " ".join([token.lemma_ for token in doc])

In [ ]:
data['text'].apply(lambda x: textCleaner(x))

0        RT @mayasolovely: As a woman you shouldn't com...
1        RT @mleew17: boy dats cold...tyga dwn bad for ...
2        RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: Y...
3        RT @C_G_Anderson: @viva_based she look like a ...
4        RT @ShenikaRoberts: The shit you hear about me...
                               ...                        
24778    you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779    you've gone and broke the wrong heart baby, an...
24780    young buck wanna eat!!.. dat nigguh like I ain...
24781                youu got wild bitches tellin you lies
24782    Ruffled | Ntac Eileen Dahlia - Beautiful color...
Name: text, Length: 24783, dtype: object

In [ ]:
X_train_, X_test, y_train_, y_test = train_test_split(
    data.index.values,
    data.label.values,
    test_size=0.10,
    random_state=42,
    stratify=data.label.values,    
)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    data.loc[X_train_].index.values,
    data.loc[X_train_].label.values,
    test_size=0.10,
    random_state=42,
    stratify=data.loc[X_train_].label.values,  
)

In [ ]:
data['data_type'] = ['not_set']*data.shape[0]
data.loc[X_train, 'data_type'] = 'train'
data.loc[X_val, 'data_type'] = 'val'
data.loc[X_test, 'data_type'] = 'test'

In [ ]:
data.groupby(['label', 'data_type']).count()

text
label data_type       
0     test         143
      train       1158
      val          129
1     test        1920
      train      15543
      val         1727
2     test         416
      train       3372
      val          375

In [ ]:
train = data.loc[data["data_type"]=="train"]
train.head()

,label,text,data_type
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,train
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,train
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,train
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,train
6,1,"!!!!!!""@__BrighterDays: I can not just sit up ...",train


In [ ]:
test = data.loc[data["data_type"]=="test"]
test.head()

,label,text,data_type
0,2,!!! RT @mayasolovely: As a woman you shouldn't...,test
12,1,""" So hoes that smoke are losers ? "" yea ... go...",test
14,1,""" bitch get up off me """,test
17,1,""" bitch who do you love """,test
25,1,""" her pussy lips like Heaven doors "" &#128524;",test


In [ ]:
val = data.loc[data["data_type"]=="val"]
val.head()

,label,text,data_type
5,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",val
27,1,""" i met that pussy on Ocean Dr . i gave that p...",val
31,1,""" i'd say im back to the old me but my old bit...",val
44,1,""" post a picture of that pussy get 200 likes """,val
46,1,""" quick piece of pussy call it a drive by """,val


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

example_text = train.loc[3, 'text']
bert_input = tokenizer(
    example_text,padding='max_length', 
    max_length = 512, 
    truncation=True, 
    return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

tensor([[  101,   106,   106,   106,   106,   106,   106,   106,   106,   106,
           155,  1942,   137,   140,   168,   144,   168,  4347,   131,   137,
           191, 12416,   168,  1359,  1131,  1440,  1176,   170,   189,  4047,
          3382,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [ ]:
example_text = tokenizer.decode(bert_input.input_ids[0])

print(example_text)

[CLS]!!!!!!!!! RT @ C _ G _ Anderson : @ viva _ based she look like a tranny [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = list(df['label'])
        self.texts =[tokenizer(
            text, 
            padding='max_length', 
            max_length = 256, 
            truncation=True,
            return_tensors="pt") 
        for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y